In [2]:
import os
import pandas as pd
import numpy as np
import plotly.io as pio
pio.renderers.default = "jupyterlab"
from plotly.subplots import make_subplots
import plotly.graph_objects as go

import analysis.data_importing as imp  # Custom importing module
import analysis.plotting as pl  # Custom plotting module
import interfaces.postprocessing as pif  # post-processing interface
import scipy.signal
import warnings

import external.custom_post.custom_post_copy as cpc

warnings.simplefilter(action='ignore', category=FutureWarning)

%load_ext autoreload
%autoreload 2

In [3]:
activity_data_dir = "data/Arnar_Larusson_2023-04-12_easy_ride"
uncleaned_data_dir = os.path.join(activity_data_dir, "uncleaned_data")
cleaned_data_dir = os.path.join(activity_data_dir, "cleaned_data")
imp.clean_all_data(uncleaned_data_dir)
clean_dfs = imp.load_cleaned_data(cleaned_data_dir)
raw_slow_df = clean_dfs["raw_slow_df"]

cleaned_data folder already exists at data/Arnar_Larusson_2023-04-12_easy_ride\cleaned_data


In [22]:
# Plot Raw Chest Signal
raw_chest = pd.DataFrame(raw_slow_df["c"]).set_index(raw_slow_df["time"])
plot_dict_chest = {
    "Raw Chest" : (raw_chest, "index", "c")
}
raw_chest_fig = pl.plot_df_columns({"Raw Chest" : (raw_chest, "index", "c")}, plottitle="Raw Chest Signal", xtitle="Time[s]", ytitle="Amplitude", show = False)

In [61]:
# Get smoothed chest signal from custom_post_copy
pp_out= pif.custom_post_format_wrapper(uncleaned_data_dir)
chest_raw = pp_out["chest_raw"]
chest_5hz = pp_out["chest_5hz"]
chest_bs = pp_out["chest_bs"]
chest_bs_smooth = pp_out["chest_bs_smooth"]
time = pp_out["time"]
X_bbyb_df = pp_out["X_bbyb_df"]
valley_peak_ALL = pp_out["valley_peak_ALL"]


In [62]:
valley_peak_ALL

array([[  102,   128],
       [  199,   239],
       [  303,   335],
       ...,
       [64053, 64084],
       [64116, 64137],
       [64164, 64241]], dtype=int64)

In [56]:
# Create plotly subplots from chest data in last cell
chest_plot_dicts = {}
x_titles = []
y_titles = []
for i,(chest_data, name) in enumerate(zip([chest_raw, chest_5hz, chest_bs, chest_bs_smooth], ["Raw Chest", "5Hz Lowpass Filtered Chest", "Bandstop Filtered Chest", "Smoothed Bandstop Filtered Chest"])):
    df = pd.DataFrame(chest_data, columns=[name]).set_index(time*60)
    chest_plot_dicts[name] = (df, "index", name)
    x_titles.append("Time[s]")
    y_titles.append("Amplitude")
    # if i == 0:
    #     plottitle = "Raw Chest Signal"
    # elif i == 1:
    #     plottitle = "5Hz Lowpass Filtered Chest Signal"
    # elif i == 2:
    #     plottitle = "Bandstop Filtered Chest Signal"
    # elif i == 3:
    #     plottitle = "Smoothed Bandstop Filtered Chest Signal"
# time_figs = pl.create_subplots(chest_plot_dicts, plottitle="Chest Data", xtitles=x_titles, ytitles=y_titles, show=False)

In [27]:
# Create spectrogram for all chest signals
raw_spec_fig = pl.create_spectrogram_fig(chest_raw, 25, "Raw")
raw_spec_fig.show(renderer = "jupyterlab")

In [36]:
# import ipywidgets as ipw
# f1 = go.FigureWidget(raw_chest_fig)
# f2 = go.FigureWidget(raw_spec_fig)
# fig_subplots = ipw.HBox([f1,f2])
# fig_subplots


    'data': [{'name': 'Raw Chest',
              'type': 'scatter',
            …

In [59]:
pl.plot_time_spec(chest_plot_dicts)

In [57]:
chest_plot_dicts["Raw Chest"][0]


,Raw Chest
0.00,17089
0.04,17075
0.08,17081
0.12,17079
0.16,17084
...,...
2573.32,17115
2573.36,17106
2573.40,17114
2573.44,17105
